In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shap

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('features.py'), '..')))

from NBAData.features import *

/Users/alexg/Documents/Documents/Prize-Picks-Prop-Predictor/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_columns', None)

s21_playoffs = pd.read_csv('../CSV_FILES/PLAYOFF_DATA/PTS_21.csv')
s21_regular = pd.read_csv('../CSV_FILES/REGULAR_DATA/PTS_21.csv')
s21 = pd.concat([s21_regular, s21_playoffs])

s22_playoffs = pd.read_csv('../CSV_FILES/PLAYOFF_DATA/PTS_22.csv')
s22_regular = pd.read_csv('../CSV_FILES/REGULAR_DATA/PTS_22.csv')
s22 = pd.concat([s22_regular, s22_playoffs])

s23_playoffs = pd.read_csv('../CSV_FILES/PLAYOFF_DATA/PTS_23.csv')
s23_regular = pd.read_csv('../CSV_FILES/REGULAR_DATA/PTS_23.csv')
s23 = pd.concat([s23_regular, s23_playoffs])

s24_playoffs = pd.read_csv('../CSV_FILES/PLAYOFF_DATA/PTS_24.csv')
s24_regular = pd.read_csv('../CSV_FILES/REGULAR_DATA/PTS_24.csv')
s24 = pd.concat([s24_regular, s24_playoffs])

data = pd.concat([s21, s22, s23, s24])
data.sort_values(by='GAME_DATE', inplace=True)

data

,Unnamed: 0,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,COMMENT,OFF_RATING,E_OFF_RATING,DEF_RATING,E_DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,EFG_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,POSS,PACE_PER40,E_USG_PCT,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE,IS_PLAYOFF,STARTING,HEIGHT,WEIGHT,GUARD,FORWARD,CENTER,TEAM_DAYS_REST,TEAM_B2B,PLAYER_DAYS_REST,PLAYER_B2B,PLAYER_MISSED_LAST,TEAM__ATL,TEAM__BKN,TEAM__BOS,TEAM__CHA,TEAM__CHI,TEAM__CLE,TEAM__DAL,TEAM__DEN,TEAM__DET,TEAM__GSW,TEAM__HOU,TEAM__IND,TEAM__LAC,TEAM__LAL,TEAM__MEM,TEAM__MIA,TEAM__MIL,TEAM__MIN,TEAM__NOP,TEAM__NYK,TEAM__OKC,TEAM__ORL,TEAM__PHI,TEAM__PHX,TEAM__POR,TEAM__SAC,TEAM__SAS,TEAM__TOR,TEAM__UTA,TEAM__WAS,OPP__ATL,OPP__BKN,OPP__BOS,OPP__CHA,OPP__CHI,OPP__CLE,OPP__DAL,OPP__DEN,OPP__DET,OPP__GSW,OPP__HOU,OPP__IND,OPP__LAC,OPP__LAL,OPP__MEM,OPP__MIA,OPP__MIL,OPP__MIN,OPP__NOP,OPP__NYK,OPP__OKC,OPP__ORL,OPP__PHI,OPP__PHX,OPP__POR,OPP__SAC,OPP__SAS,OPP__TOR,OPP__UTA,OPP__WAS,MATCHUP_AVG_PTS_LAST_3,GAMES_VS_OPP,MIN_ROLL_AVG_2,FGA_ROLL_AVG_2,FGM_ROLL_AVG_2,FG_PCT_ROLL_AVG_2,FG3A_ROLL_AVG_2,FG3M_ROLL_AVG_2,FG3_PCT_ROLL_AVG_2,FTM_ROLL_AVG_2,FTA_ROLL_AVG_2,FT_PCT_ROLL_AVG_2,USG_PCT_ROLL_AVG_2,TS_PCT_ROLL_AVG_2,EFG_PCT_ROLL_AVG_2,TEAM_PACE_ROLL_AVG_2,TEAM_OFF_RATING_ROLL_AVG_2,OPP_DEF_RATING_ROLL_AVG_2,MIN_ROLL_AVG_4,FGA_ROLL_AVG_4,FGM_ROLL_AVG_4,FG_PCT_ROLL_AVG_4,FG3A_ROLL_AVG_4,FG3M_ROLL_AVG_4,FG3_PCT_ROLL_AVG_4,FTM_ROLL_AVG_4,FTA_ROLL_AVG_4,FT_PCT_ROLL_AVG_4,USG_PCT_ROLL_AVG_4,TS_PCT_ROLL_AVG_4,EFG_PCT_ROLL_AVG_4,TEAM_PACE_ROLL_AVG_4,TEAM_OFF_RATING_ROLL_AVG_4,OPP_DEF_RATING_ROLL_AVG_4,MIN_ROLL_AVG_6,FGA_ROLL_AVG_6,FGM_ROLL_AVG_6,FG_PCT_ROLL_AVG_6,FG3A_ROLL_AVG_6,FG3M_ROLL_AVG_6,FG3_PCT_ROLL_AVG_6,FTM_ROLL_AVG_6,FTA_ROLL_AVG_6,FT_PCT_ROLL_AVG_6,USG_PCT_ROLL_AVG_6,TS_PCT_ROLL_AVG_6,EFG_PCT_ROLL_AVG_6,TEAM_PACE_ROLL_AVG_6,TEAM_OFF_RATING_ROLL_AVG_6,OPP_DEF_RATING_ROLL_AVG_6,PLAYER_HOME_AVG_PTS,PLAYER_AWAY_AVG_PTS,PTS_LAG_1,PTS_LAG_2,PTS_LAG_3,PTS_LAG_4,DEF_CATEGORY,PTS_VS_DEF_STRONG,PTS_VS_DEF_WEAK,PTS_VS_DEF_DIFF,FGA_VS_DEF_STRONG,FGA_VS_DEF_WEAK,FGA_VS_DEF_DIFF,FTA_VS_DEF_STRONG,FTA_VS_DEF_WEAK,FTA_VS_DEF_DIFF,FG3A_VS_DEF_STRONG,FG3A_VS_DEF_WEAK,FG3A_VS_DEF_DIFF,USG_PCT_VS_DEF_STRONG,USG_PCT_VS_DEF_WEAK,USG_PCT_VS_DEF_DIFF,TOV_VS_DEF_STRONG,TOV_VS_DEF_WEAK,TOV_VS_DEF_DIFF,TEAM_STARTER_OFF_RATING_AVG,TEAM_STARTER_DEF_RATING_AVG,TEAM_STARTER_USG_PCT_AVG,NUM_USUAL_STARTERS_PRESENT,TEAM_ID_OPP,NUM_USUAL_STARTERS_PRESENT_OPP,TEAM_ID_OPP.1,OPP_STARTER_AVG_DEF_RATING,TEAM_ID_OPP.2,OPP_GUARDS_AVG_DEF_RATING_OPP,TEAM_ID_OPP.3,OPP_FORWARDS_AVG_DEF_RATING_OPP,TEAM_ID_OPP.4,OPP_CENTERS_AVG_DEF_RATING_OPP,TEAM_STARTER_SPACING_METRIC,TEAM_STARTER_PACE,TEAM_ID_OPP.5,OPP_STARTER_PACE,PACE_EXPECTATION,IS_STAR,NUM_STARS_ON_TEAM,MATCHUP_KEY,SERIES,GAME_IN_SERIES,SERIES_NAME
7052,7052,Dwight Howard,2730,PHI vs. WAS,PHI,1610612755,WAS,1,22000013,2020-12-23,W,13,4,2,10,2,2,1.000,0,0,NaN,0,0,NaN,3,7,0,1,2,3,8,20.0,2.000,1.000000,NaN,NaN,100.0,99.3,71.0,72.1,29.0,0.200,0.438,0.323,0.222,1.000,1.00,0.114,1.000,110.68,111.19,0.263,30.0,92.66,0.120,"DEC 23, 2020",240,41,87,0.471,8,28,0.286,23,30,0.767,10,37,47,22,11,8,16,25,113,106.403013,7.0,4.0,40.0,0.458824,1.610613e+09,106.403013,106.20,107.66,109.12,0,0,6-10,265.0,0,1,1,3.0,0,3.0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,4.00,1,13.0,2.0,2.0,1.00,0.0,0.0,1.00,0.0,0.0,1.00,0.11,1.00,1.00,10

### Features that will be used to train my XGBoost model

In [5]:
features = [
    #player/team stats
    'MIN', 'FGA', 'FTA', 'FG3A', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'REB', 'OREB', 'DREB', 'STL', 'BLK', 'TOV', 'PF',
    'OFF_RATING', 'DEF_RATING', 'PointsPerShot', 'EFG_PCT', 'AST_PCT', 'AST_TOV', 'USG_PCT', 'TS_PCT', 'PACE', 'POSS',
    'TEAM_FGA', 'TEAM_FG3A', 'TEAM_FG_PCT', 'TEAM_FG3_PCT', 'TEAM_AST', 'TEAM_REB', 'TEAM_STL', 'TEAM_BLK', 'TEAM_PACE', 'TEAM_PTS',
    #grab oppenents avgerages for the game
    'OPP_PACE', 'OPP_DEF_RATING','OPP_STL', 'OPP_BLK', 'OPP_REB', 'OPP_FG_PCT',
    #rolling averages
    'MIN_ROLL_AVG_2', 'FGA_ROLL_AVG_2', 'FGM_ROLL_AVG_2', 'FG_PCT_ROLL_AVG_2',
    'FG3A_ROLL_AVG_2', 'FG3M_ROLL_AVG_2', 'FG3_PCT_ROLL_AVG_2', 'FTM_ROLL_AVG_2',
    'FTA_ROLL_AVG_2', 'FT_PCT_ROLL_AVG_2', 'USG_PCT_ROLL_AVG_2', 'TS_PCT_ROLL_AVG_2',
    'EFG_PCT_ROLL_AVG_2', 'TEAM_PACE_ROLL_AVG_2', 'TEAM_OFF_RATING_ROLL_AVG_2',
    'OPP_DEF_RATING_ROLL_AVG_2', 'MIN_ROLL_AVG_4', 'FGA_ROLL_AVG_4', 'FGM_ROLL_AVG_4',
    'FG_PCT_ROLL_AVG_4', 'FG3A_ROLL_AVG_4', 'FG3M_ROLL_AVG_4', 'FG3_PCT_ROLL_AVG_4',
    'FTM_ROLL_AVG_4', 'FTA_ROLL_AVG_4', 'FT_PCT_ROLL_AVG_4', 'USG_PCT_ROLL_AVG_4',
    'TS_PCT_ROLL_AVG_4', 'EFG_PCT_ROLL_AVG_4', 'TEAM_PACE_ROLL_AVG_4',
    'TEAM_OFF_RATING_ROLL_AVG_4', 'OPP_DEF_RATING_ROLL_AVG_4', 'MIN_ROLL_AVG_6',
    'FGA_ROLL_AVG_6', 'FGM_ROLL_AVG_6', 'FG_PCT_ROLL_AVG_6', 'FG3A_ROLL_AVG_6',
    'FG3M_ROLL_AVG_6', 'FG3_PCT_ROLL_AVG_6', 'FTM_ROLL_AVG_6', 'FTA_ROLL_AVG_6',
    'FT_PCT_ROLL_AVG_6', 'USG_PCT_ROLL_AVG_6', 'TS_PCT_ROLL_AVG_6', 'EFG_PCT_ROLL_AVG_6',
    'TEAM_PACE_ROLL_AVG_6', 'TEAM_OFF_RATING_ROLL_AVG_6', 'OPP_DEF_RATING_ROLL_AVG_6',
    'PLAYER_HOME_AVG_PTS', 'PLAYER_AWAY_AVG_PTS',
    #player team
    'TEAM__ATL', 'TEAM__BKN', 'TEAM__BOS', 'TEAM__CHA', 'TEAM__CHI',
    'TEAM__CLE', 'TEAM__DAL', 'TEAM__DEN', 'TEAM__DET', 'TEAM__GSW',
    'TEAM__HOU', 'TEAM__IND', 'TEAM__LAC', 'TEAM__LAL', 'TEAM__MEM',
    'TEAM__MIA', 'TEAM__MIL', 'TEAM__MIN', 'TEAM__NOP', 'TEAM__NYK',
    'TEAM__OKC', 'TEAM__ORL', 'TEAM__PHI', 'TEAM__PHX', 'TEAM__POR',
    'TEAM__SAC', 'TEAM__SAS', 'TEAM__TOR', 'TEAM__UTA', 'TEAM__WAS',
    #opponent team
    'OPP__ATL', 'OPP__BKN', 'OPP__BOS', 'OPP__CHA', 'OPP__CHI',
    'OPP__CLE', 'OPP__DAL', 'OPP__DEN', 'OPP__DET', 'OPP__GSW',
    'OPP__HOU', 'OPP__IND', 'OPP__LAC', 'OPP__LAL', 'OPP__MEM',
    'OPP__MIA', 'OPP__MIL', 'OPP__MIN', 'OPP__NOP', 'OPP__NYK',
    'OPP__OKC', 'OPP__ORL', 'OPP__PHI', 'OPP__PHX', 'OPP__POR',
    'OPP__SAC', 'OPP__SAS', 'OPP__TOR', 'OPP__UTA', 'OPP__WAS',
    #player vs defense
    'DEF_CATEGORY',
    'PTS_VS_DEF_STRONG', 'PTS_VS_DEF_WEAK', 'PTS_VS_DEF_DIFF',
    'FGA_VS_DEF_STRONG', 'FGA_VS_DEF_WEAK', 'FGA_VS_DEF_DIFF',
    'FTA_VS_DEF_STRONG', 'FTA_VS_DEF_WEAK', 'FTA_VS_DEF_DIFF',
    'FG3A_VS_DEF_STRONG', 'FG3A_VS_DEF_WEAK', 'FG3A_VS_DEF_DIFF',
    'USG_PCT_VS_DEF_STRONG', 'USG_PCT_VS_DEF_WEAK', 'USG_PCT_VS_DEF_DIFF',
    'TOV_VS_DEF_STRONG', 'TOV_VS_DEF_WEAK', 'TOV_VS_DEF_DIFF',
    #player specific
    'MATCHUP_AVG_PTS_LAST_3', 'GUARD', 'FORWARD', 'CENTER', 'STARTING', 'HOME_GAME', 
    'PLAYER_DAYS_REST', 'PLAYER_B2B', 'PLAYER_MISSED_LAST', 'IS_STAR',
    #Starter features
    'TEAM_STARTER_OFF_RATING_AVG',	'TEAM_STARTER_DEF_RATING_AVG', 'TEAM_STARTER_USG_PCT_AVG', 'NUM_USUAL_STARTERS_PRESENT',
    'TEAM_STARTER_SPACING_METRIC', 'TEAM_STARTER_PACE',
    #OPP Starter features
    'NUM_USUAL_STARTERS_PRESENT_OPP', 'OPP_STARTER_AVG_DEF_RATING', 'OPP_GUARDS_AVG_DEF_RATING_OPP',
    'OPP_FORWARDS_AVG_DEF_RATING_OPP', 'OPP_CENTERS_AVG_DEF_RATING_OPP', 'OPP_STARTER_PACE', 'PACE_EXPECTATION',
    #playoff features
    'NUM_STARS_ON_TEAM', 'IS_PLAYOFF', 'SERIES', 'GAME_IN_SERIES',
]

In [28]:
corr_matrix = data.corr(numeric_only=True)
target = corr_matrix['PTS'].sort_values(ascending=False).tail(15)
target


GameInSeries                 -0.015548
NUM_USUAL_STARTERS_PRESENT   -0.015817
CENTER                       -0.017195
DEF_CATEGORY                 -0.019576
OPP_STL                      -0.019891
TEAM_TOV                     -0.021359
PACE_PER40                   -0.025189
PACE                         -0.025189
TOV_VS_DEF_DIFF              -0.026566
TEAM_B2B                     -0.037901
OPP_REB                      -0.044329
FORWARD                      -0.049670
PLAYER_MISSED_LAST           -0.053209
PLAYER_DAYS_REST             -0.053274
PLAYER_ID                    -0.117134
Name: PTS, dtype: float64

## Train XGBoost model

In [6]:
from xgboost_model import *

X = data[features]
y = data['PTS']
modelPTS = train_xgb_model(X, y, stat_line='PTS')
modelPTS

Fitting 3 folds for each of 50 candidates, totalling 150 fits

Model Performance Metrics for PTS:
R2 Score: 0.9991
MAE: 0.1697
RMSE: 0.2633

Best Parameters: {'subsample': 0.8, 'reg_lambda': 10, 'reg_alpha': 1, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.7}
Model saved to Models/PTS_xgb_model.pkl


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [7]:
import shap

# Create SHAP explainer
explainer = shap.TreeExplainer(modelPTS)
shap_values = explainer.shap_values(X)

# Get feature importance scores
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': np.abs(shap_values).mean(axis=0)
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nTop 15 Most Important Features For XGBoost Model:")
print(feature_importance.head(15))



Top 15 Most Important Features For XGBoost Model:
                    feature  importance
1                       FGA    3.924093
169  MATCHUP_AVG_PTS_LAST_3    1.627957
16            PointsPerShot    1.552184
21                   TS_PCT    0.933207
2                       FTA    0.809583
4                    FG_PCT    0.240734
20                  USG_PCT    0.205887
5                    FT_PCT    0.183417
0                       MIN    0.140606
23                     POSS    0.135479
17                  EFG_PCT    0.125665
3                      FG3A    0.073983
6                   FG3_PCT    0.068198
12                      TOV    0.047894
89      PLAYER_AWAY_AVG_PTS    0.018592


In [8]:
import shap

# Create SHAP explainer
explainer = shap.TreeExplainer(modelPTS)
shap_values = explainer.shap_values(X)

# Get feature importance scores
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': np.abs(shap_values).mean(axis=0)
})

less_important_features = feature_importance[feature_importance['importance'] < 0.001]
print("\nTop 15 Least Important Features For XGBoost Model:")
print(less_important_features.head(15))


Top 15 Least Important Features For XGBoost Model:
         feature  importance
9           DREB    0.000110
10           STL    0.000053
11           BLK    0.000013
13            PF    0.000113
15    DEF_RATING    0.000428
18       AST_PCT    0.000350
22          PACE    0.000129
25     TEAM_FG3A    0.000178
26   TEAM_FG_PCT    0.000300
27  TEAM_FG3_PCT    0.000198
28      TEAM_AST    0.000238
30      TEAM_STL    0.000067
31      TEAM_BLK    0.000096
32     TEAM_PACE    0.000142
33      TEAM_PTS    0.000501
